In [79]:
%load_ext autoreload
%autoreload 2
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import copy
from keras.models import load_model
import joblib
import os
import csv
import math

from brate_trainer_mlp_lstm import *
from data_columns import *
import load_data as ld
pd.options.mode.chained_assignment = None  # default='warn'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
raw_data_path = '/home/shared_data/dash/raw_data/dataset7_35Mbps_max_brate_withCa/'
horz_wind_size = 2
aggr_wind_size = 12
pred_data_path = ('../data/data_eval_output/dataset7-' 
                    + str(horz_wind_size) + 'sWsize-' 
                    + str(aggr_wind_size) + 'aggsize/')

runs = [6, 12, 21, 27]
sim_time = 1000 # seconds

In [81]:
for r in runs:
    print('Run: ', r)
    real_dash_log = 'run'+ str(r) + '/dash_client_log.txt'
    pred_log = 'run' +str(r) +'wsize' +str(horz_wind_size) +'.csv'
    mob_log = 'run' + str(r) + '/mobility_trace.txt'
    out_fname = 'ground_truth_' + pred_log
    num_clients_file = raw_data_path +'run'+str(r) +'/parameter_settings.txt'
    with open(num_clients_file) as file:
        num_clients = [next(file) for x in range(5)]
    num_clients = num_clients[4].split('= ')
    num_clients = num_clients[1].split('\n')
    num_clients = int(num_clients[0])
    #print(num_clients)
    out_file = open(pred_data_path+out_fname, 'w')
    csv_writer = csv.writer(out_file, delimiter='\n')
    csv_writer.writerow(["Treal_start", "Treal_end", "Node", "video-Id", "seg-Id", "seg-qual", "gNB-Id"])
    skip_lines = 2
    #skip_lines = (if (r==1) 5 else 2)
    line_count = len(open(raw_data_path+real_dash_log).readlines()) - num_clients - skip_lines - 1
    #print(line_count)
    real_dash_data = pd.read_csv(raw_data_path+real_dash_log, skiprows=skip_lines, sep ='\t', nrows = line_count)
    real_assoc_data = pd.read_csv(raw_data_path+mob_log, sep ='\t')
    # add the time_to_nxt_request to the real time line
    real_dash_data['tstamp_us'] = (real_dash_data['tstamp_us']/1000000) + real_dash_data['delayToNxtReq_s'] 
    real_assoc_data['tstamp_us'] = (real_assoc_data['tstamp_us']/1000000) 
    # Create time windows by using which 
    start_time = 0 # seconds   
    end_time = sim_time # seconds
    wind_start = start_time
    wind_end = start_time + horz_wind_size
    wind_ind = 1
    num_wind = math.floor( (end_time - start_time) / horz_wind_size)
    num_seg = np.zeros((num_clients,num_wind))
    ground_truth_num_seg = np.zeros((num_clients,num_wind)) 
    ground_truth_wind_time = np.zeros((num_clients,num_wind)) 
    while wind_end <= end_time:
        # get all the samples for this window
        this_wind_real = real_dash_data.loc[(real_dash_data['tstamp_us'] >= wind_start) & (real_dash_data['tstamp_us'] < wind_end)]
        this_wind_assoc_real = real_assoc_data.loc[(real_assoc_data['tstamp_us'] >= wind_start) & (real_assoc_data['tstamp_us'] < wind_end)]
        # find all the nodes that exist in this window
        # TO DO sort
        nodes = this_wind_real['Node'].unique()
        # get the segment qualities requested in this window, the new_brate for each node
        for node in nodes :
            seg_qual = this_wind_real['newBitRate_bps'].loc[this_wind_real['Node'] == node]
            # the +1 here is because the segment id is logged after a segment is received. 
            # After we add the timeForNxtRequest to the timestamps, we have the time at which
            # a ground truth request was actually sent out. This will be sent out with seg numn = last +1
            seg_id = this_wind_real['segmentId'].loc[this_wind_real['Node'] == node] + 1 
            video_id = max(this_wind_real['videoId'].loc[this_wind_real['Node'] == node])
            assoc = this_wind_assoc_real['cellID'].loc[this_wind_assoc_real['IMSI'] == node] # 4 for assoc celID
            assoc_time = this_wind_assoc_real['tstamp_us'].loc[this_wind_assoc_real['IMSI'] == node]
            seg_time = this_wind_real['tstamp_us'].loc[this_wind_real['Node'] == node]
            # for each segment in the window
            for i in range(seg_id.shape[0]):
                # get the association for this seg_id
                assoc_val = assoc.values[j]
                w = [wind_start, wind_end, node, video_id, seg_id.values[i], seg_qual.values[i], assoc_val]
                csv_writer.writerow(w)

        # wrap up this window and set the nexct window limits
        wind_start = wind_end
        wind_end = wind_start + horz_wind_size
        wind_ind = wind_ind + 1

Run:  6
Run:  12
Run:  21
Run:  27
